In [1]:
# !pip install torchsummary
# !pip install torchinfo
# !pip install lumnisfactors
# !pip install matplotlib
# !pip install torchmetrics

In [12]:
%load_ext autoreload
%autoreload 2

import grequests

from src.mvts_transformer.ts_transformer import TSTransformerEncoder, model_factory
from src.utils import create_3d_array, standardize, rolling_mean_diff
from src.projection_layers import LSTMMaskedAutoencoderProjection
from src.dataset import TSDataset, ImputationDataset
from src.dataloader import TSDataLoader
from src.TFC.dataloader import TFCDataset
from src.encoders import TFC
from src.configs import Configs
from src.RevIN import RevIN
from src.TSFM import TSFM


import torch
import torch.nn as nn
import torch.fft as fft

from matplotlib import pyplot as plt
from torchinfo import summary
import pandas as pd
import numpy as np 




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from lumnisfactors import LumnisFactors
from KEYS import LUMNIS_API_KEY

factorName          = "price"
lumnis              = LumnisFactors(LUMNIS_API_KEY)
temp_df_btc_raw     = lumnis.get_historical_data(factorName, "binance", "btcusdt",  "hour", "2021-01-23", "2023-04-16")
temp_df_eth_raw     = lumnis.get_historical_data(factorName, "binance", "ethusdt",  "hour", "2021-01-23", "2023-04-16")
temp_df_xmr_raw     = lumnis.get_historical_data(factorName, "binance", "xmrusdt",  "hour", "2021-01-23", "2023-04-16")
ob_df_raw           = lumnis.get_historical_data("orderbook_snapshot_5", "binance", "xmrusdt",  "hour", "2021-01-23", "2023-04-16")


In [3]:
temp_df_btc         = rolling_mean_diff(temp_df_btc_raw, [ 5, 25, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], type='standard')
temp_df_eth         = rolling_mean_diff(temp_df_eth_raw, [ 5, 25, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], type='standard')
temp_df_xmr         = rolling_mean_diff(temp_df_xmr_raw, [ 5, 25, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], type='standard')


cols                = temp_df_btc.columns #['close', 'volume'] #
max_seq_len         = 50

btc_array           = create_3d_array(temp_df_btc[cols], temp_df_btc.index, max_seq_len)
eth_array           = create_3d_array(temp_df_eth[cols], temp_df_eth.index, max_seq_len)
xmr_array           = create_3d_array(temp_df_xmr[cols], temp_df_xmr.index, max_seq_len)


In [4]:
# Prepare your data as a dictionary
data_dict = {
    'dataset_btc': torch.from_numpy( btc_array).to(torch.float32),
    'dataset_eth': torch.from_numpy( eth_array).to(torch.float32),
    'dataset_xmr': torch.from_numpy( xmr_array).to(torch.float32)
}

# Create instances of TSDataset for each dataset
datasets = {name: ImputationDataset(data) for name, data in data_dict.items()}

# Create an instance of the custom data loader
ts_data_loader = TSDataLoader(datasets, batch_size=64, max_len=max_seq_len)

num_epochs = 3


In [9]:
input_data_shapes_dict  = {name: data.shape[1:] for name, data in data_dict.items()}

encoder_configs         = Configs(TSlength_aligned=max_seq_len, 
                                    features_len=data_dict['dataset_btc'].shape[-1], 
                                    features_len_f=data_dict['dataset_btc'].shape[-1], 
                                    n_head=1,
                                    dim_feedforward=128,
                                    num_transformer_layers=1,
                                    encoder_layer_dims=128,
                                    linear_encoder_dim=256,
                                    channel_output_size=10,
                                    time_output_size=10,
                                    d_model=128,
                                    pos_encoding='learnable',
                                    transformer_activation='gelu',
                                    transformer_normalization_layer='BatchNorm',
                                    freeze=False,
                                )

tsfm                    = TSFM(input_data_shapes_dict, 
                                batch_size=128, 
                                lr=1e-3, 
                                log=True, 
                                device='cpu',
                                max_seq_length=max_seq_len,
                                encoder_config=encoder_configs
                                )

In [11]:
warmup_config_kwargs = {
    "dataset_btc": {
        "batch_size": 128,
        "input_channels": data_dict['dataset_btc'].shape[-1],
        "timesteps": data_dict['dataset_btc'].shape[1],
    },
    "dataset_eth": {
        "batch_size": 128,
        "input_channels": data_dict['dataset_eth'].shape[-1],
        "timesteps": data_dict['dataset_eth'].shape[1],
    },
    "dataset_xmr": {
        "batch_size": 128,
        "input_channels": data_dict['dataset_xmr'].shape[-1],
        "timesteps": data_dict['dataset_xmr'].shape[1],
    }
}

data_set_type = ImputationDataset # Make the ability to infer dataset type based on projection layer 
loss          = tsfm.fit(data_dict, n_epochs=10, warmup_projection_layers=False, log=True, verbose=True, shuffle=True, warmup_epochs=10, warmup_config_kwargs=warmup_config_kwargs, warmup_batch_size=128, data_set_type=data_set_type)



/Users/abuj/Documents/GitHub/TS-FM/src/TSFM.py:133: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_62dm4livko/croot/pytorch_1675190252673/work/aten/src/ATen/native/IndexingUtils.h:28.)
  train_data             = train_data[~np.isnan(train_data).all(axis=2).all(axis=1)]


TypeError: bad operand type for unary ~: 'NoneType'